In [1]:
from __future__ import print_function

In [2]:
import sys
sys.path.append('../build/')

In [3]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [4]:
import versor as vsr

In [5]:
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, interact, widgets
from traitlets import link, dlink

# Drawing functions

In [144]:
def vector_mesh(vec, position=None, color='black', linewidth=2, arrow=False):
    length = vec.norm()
    line = Line(geometry=PlainGeometry(vertices=[[0,0,0], [0,length,0]],
                                   colors = [color, color]), 
            material=LineBasicMaterial(linewidth=linewidth,
                                       vertexColors='VertexColors'), 
            type='LinePieces')
    if arrow:
        cone = Mesh(geometry=CylinderGeometry(radiusTop = 0,
                                              radiusBottom = 0.1,
                                              height = 0.2,
                                              radiusSegments = 20,
                                              heightSegments = 1,
                                              openEnded = False),
                    material=LambertMaterial(color=color))
        cone.position = [0,length-0.1,0]
        line.children = [cone]
    line.quaternion = np.array(vec.unit().ratio(vsr.Vec(0,1,0)).quat()).tolist()
    if position is not None:
        line.position = np.array(position)[:3].tolist()
    return line

def bivector_mesh(biv, position=None, as_plane=False, color='gray'):
    
    weight = biv.norm()
    if as_plane:
        geometry = PlaneGeometry(width=weight, height=weight)
    else:
        geometry = CircleGeometry(radius=weight,
                                  segments=64)
        
    material = LambertMaterial(color=color)
    material.transparent = True
    material.opacity = 0.75
    mesh = Mesh(geometry=geometry,
                material=material)
    mesh.quaternion = np.array(biv.unit().duale().ratio(vsr.Vec(0,0,1)).quat()).tolist()
    if position is not None:
        mesh.position = np.array(position)[:3].tolist()
    return mesh

def trivector_mesh(tri, color='gray'):
    geometry = BoxGeometry()
    material = LambertMaterial(color=color)
    mesh = Mesh(geometry=geometry,
                material=material)
    mesh.material.wireframe = True
    mesh.position = [0.5,0.5,0.5]

    return mesh

def sphere_mesh(sph, color='violet'):
    if type(sph) == vsr.Dls:
        sph = sph.undual()
    radius = sph.radius()
    geometry = SphereGeometry(radius=radius,
                            widthSegments=64,
                            heightSegments=64) 
    material = LambertMaterial(color=color)
    material.transparent = True
    material.opacity = 0.75
    mesh = Mesh(geometry=geometry, material=material)
    mesh.position=np.array(sph.pnt())[:3].tolist()
    return mesh

def point_mesh(pnt, color='gray'):
    mesh = Mesh(geometry=SphereGeometry(radius=0.05,
                                        widthSegments=32,
                                        heightSegments=32), 
              material=LambertMaterial(color=color))
    mesh.position = np.array(pnt)[:3].tolist()
    return mesh

def line_mesh(line, arrow=True, length=100, color='gray'):
    v = np.array(line.loc(vsr.Vec(0,0,0).null()))[:3]
    d = np.array(line.dir()) * 0.5 * length
    linesgeom = PlainGeometry(vertices=[(v + d).tolist(), (v - d).tolist()],
                                        colors = [color, color])
    mesh = Line(geometry=linesgeom, 
                material=LineBasicMaterial(linewidth=2, vertexColors='VertexColors'), 
                type='LinePieces')
    if arrow:
        cone = Mesh(geometry=CylinderGeometry(radiusTop = 0,
                                              radiusBottom = 0.04,
                                              height = 0.2,
                                              radiusSegments = 20,
                                              heightSegments = 1,
                                              openEnded = False),
                    material=LambertMaterial(color=color))
        cone.position = v.tolist()
        cone.quaternion = np.array(line.dir().vec().unit().ratio(vsr.Vec(0,1,0)).quat()).tolist()
        mesh.children = [cone]
    return mesh

def plane_mesh(plane, width=10, height=10, color='gray'):
    mesh = Mesh(geometry=PlaneGeometry(width=width, height=height),
               material=LambertMaterial(color=color,
                                        transparent=True,
                                        opacity=0.75))
    mesh.position = np.array(plane.loc(vsr.Vec(0,0,0).null()))[:3].tolist()
    if type(plane) == vsr.Pln:
        normal = plane.dual().dir().vec()
    else:
        normal = plane.dir().vec()
    mesh.quaternion = np.array(normal.ratio(vsr.Vec(0,0,1)).quat()).tolist()
    return mesh

def circle_mesh(circle, color='gray'):
    mesh = Mesh(geometry = RingGeometry(innerRadius=cir.radius()-0.01,
                                        outerRadius=cir.radius()+0.01,
                                        thetaSegments=64), 
                material= LambertMaterial(color=color),
                position=np.array(cir.pnt())[:3].tolist(),
                quaternion = cir.rot().quat().tolist()
               )
    return mesh
    
def frame_mesh(position=None, quaternion=None, size=1, linewidth=2):
    frame = Line(geometry=PlainGeometry(vertices=[[0, 0, 0],[size, 0, 0],
                                                  [0, 0, 0],[0, size, 0],
                                                  [0, 0, 0],[0, 0, size]],
                                        colors = ['red','red', 
                                                  'green','green', 
                                                  'blue','blue']), 
             material=LineBasicMaterial(linewidth=linewidth, vertexColors='VertexColors'), 
             type='LinePieces')
    if position is not None:
        frame.position = position
    if quaternion is not None:
        frame.quaternion = quaternion
    return frame

In [140]:
a = vsr.Vec(1,0,2).null()
b = vsr.Vec(0,1,1).null()
c = vsr.Vec(0,-1,1).null()
dll1 = vsr.Dll(a,b)
dll2 = vsr.Dll(b,c)
dll3 = vsr.Dll(c,a)

pln = vsr.Pln(a,b,c)
cir = vsr.Cir(a,b,c)
cir.surround()

Dls: [ 0.25 0 1.25 1 0.25 ]

In [145]:
scene = Scene(children=[
        frame_mesh(),
#         point_mesh(a, color='lawngreen'),
#         point_mesh(b, color='skyblue'),
#         point_mesh(c),
        line_mesh(dll1,color='hotpink',arrow=False),
        line_mesh(dll2,color='skyblue'),
        line_mesh(dll3,color='skyblue'),
#         sphere_mesh(cir.surround()),
#         plane_mesh(pln, width=5, height=5),
        circle_mesh(cir, color='lightsalmon'),
        AmbientLight(color='#cccccc')])
camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

In [9]:
def rgb2hex(r, g, b):
    return "#{0:02x}{1:02x}{2:02x}".format(r, g, b)
print(rgb2hex(255,0,0))

#ff0000


In [10]:
def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb
rgb_to_hex((0,0,1))

'000001'

# Figures

## Duality

In [11]:
B = vsr.Biv(1,0,0)
a = B.duale()

scene = Scene(children=[
        frame_mesh(),
        vector_mesh(a,color='blue', arrow=True),
        bivector_mesh(B,as_plane=False,color='hotpink'),
#         DirectionalLight(color='#ccaabb', position=[1,1,1]),
        AmbientLight(color='#cccccc')])
camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'

display(renderer)

## Projection and Rejection

In [12]:
a = vsr.Vec(1,1,1)
B = vsr.Biv(1,0,0)
c = a.project_onto(B)
d = a.reject_from(B)

scene = Scene(children=[
        frame_mesh(),
        vector_mesh(a,color='gray', arrow=True),
        vector_mesh(c,color='gray'),
        vector_mesh(d,position=np.array(c).tolist(),color='gray'),
        bivector_mesh(B,color='hotpink'),
#         DirectionalLight(color='#ccaabb', position=[1,1,1]),
        AmbientLight(color='#cccccc')])
camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

## Reflection

In [13]:
a = vsr.Vec(1,0,1)
B = vsr.Biv(2,0,0)
n = B.duale().unit()
a_reflected = a.reflect_in_line(n)
a_reflected2 = a.reflect_in_plane(n)

children = [
    frame_mesh(),
    vector_mesh(a,color='black', arrow=True),
    vector_mesh(a_reflected,color='black', arrow=True),
    vector_mesh(a_reflected2,color='black', arrow=True),
#     vector_mesh(n,color='black', arrow=True),
    bivector_mesh(B, as_plane=True,color='hotpink')]

children[-1].material.transparent = True
children[-1].material.opacity = 0.75


def show(children):
    scene = Scene(children= children + [AmbientLight(color='#cccccc')])
    camera = PerspectiveCamera(position=[1, -3, 0])
    renderer = Renderer(camera=camera, 
                        scene=scene, 
                        controls=[TrackballControls(controlling=camera)],
                        background = 'white')
    return renderer
display(show(children))

## Rotation

In [14]:
n1 = vsr.Vec(0,1,0).unit()
n2 = n1.spin(vsr.Biv(-np.deg2rad(30) / 2,0,0).exp())
B = n1 ^ n2
R = n1 * n2
a = vsr.Vec(1,0,1).unit()
a_rot = a.reflect_in_line(n2).reflect_in_line(n1)

B_mesh = bivector_mesh(B,color='gray')
B_mesh.material.opacity = 0.9
B_mesh.material.transparent = True

scene = Scene(children=[
#         frame_mesh(),
        vector_mesh(n1,color='yellow'),
        vector_mesh(n2,color='cyan'),
        vector_mesh(a, color='hotpink'),
        vector_mesh(a.reflect_in_line(n2)),
        vector_mesh(a_rot, color='black'),
#         bivector_mesh(n1.unduale(),color='yellow'),
#         bivector_mesh(n2.unduale(),color='cyan'),
        B_mesh,
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

# Conformal Geometric Algebra

In [15]:
p1 = vsr.Vec(1,0,1).null()
p2 = vsr.Vec(2,1,1).null()
l = vsr.Dll(p1,p2)
l_dir = l.dir().vec().unit()
l_loc = l.loc(vsr.Vec(0,0,0).null()).vec()

d = l_loc <= l_dir.unduale()

scene = Scene(children=[
        frame_mesh(),
        point_mesh(p1,color='#ff00ff'),
        vector_mesh(p1.vec(),color='#ff00ff'),
#         make_text('p1', position=(np.array(p1)[:3] + np.array([0,0,0.5]) ).tolist(), height=0.5),
        point_mesh(p2,color='#00ffff'),
        vector_mesh(l_dir,position=p1,color='#ffff00', arrow=True),
        bivector_mesh(l_dir.unduale(), position=p1, color='#ffff00'),
        vector_mesh(d,position=p1,color='gray'),


        line_mesh(l, length=100),
        
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

### Lines

In [39]:
p1 = vsr.Vec(1,0,1).null()
p2 = vsr.Vec(2,1,1).null()
l1 = vsr.Dll(vsr.Vec(0,0,0), vsr.Vec(1,0,0)).unit()
l = vsr.Dll(p1,p2).unit()
l_dir = l.dir().vec().unit()
l_loc = l.loc(vsr.Vec(0,0,0).null()).vec()

l2 = l - l1
l3 = l.comm(l1)
print(l3)

m1 = l1.ratio(l,0.1)
# print(l)
# print(l1.spin(m1))
# ms = [m1]
# for i in range(9):
#     m1 = m1 * m1
#     ms.append(m1)
    
# print(ms)
    

# line_meshes = [line_mesh(l) for ] 

m = ((l * l1.inv()).log() * 0.5 ).exp()
w = m.trs().reject_from(m.rot().log().unit())
v = m.trs().project_onto(m.rot().log().unit())
print(w)

d = l_loc <= l_dir.unduale()

scene = Scene(children=[
        frame_mesh(),
        line_mesh(l1.spin(m1)),
        line_mesh(l1.spin(m1 * m1)),
        line_mesh(l1.spin(m1 * m1 * m1)),
        line_mesh(l1.spin(m1 * m1 * m1 * m1)),
        line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1)),
        line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1)),
        line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1 * m1 )),
        line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1 * m1 * m1)),
        line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1)),
        line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1)),
        line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1)),
        

        line_mesh(l1, color='#ffff00', length=100),
        point_mesh(l_loc, color=rgb2hex(255,56,0)),
        point_mesh(vsr.Vec(0,0,0),color='#ffff00'),
        point_mesh(p1,color='#ff00ff'),
        vector_mesh(w, position=v),
        vector_mesh(v),
        
        line_mesh(l2, color='hotpink', length=100),
        line_mesh(l3, color=rgb2hex(255,155,0), length=100),


        vector_mesh(p1.vec(),color='#ff00ff'),
        vector_mesh(l_dir,position=p1,color='#ff00ff', arrow=True),


        line_mesh(l,color='#ff00ff', length=100),
        
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

Dll: [ 0.7071 0 0 0 -0.7071 0.7071 ]
Vec: [ 0 0 1 ]


In [40]:
print(m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1)

Mot: [ 0.92 -0.38 0 0 -5.2e-18 0.38 -0.46 0.19 ]


In [59]:
l1 = vsr.Dll(vsr.Vec(0,0,0), vsr.Vec(1,0,0)).unit()
l2 = vsr.Dll(vsr.Vec(0,-1,1), vsr.Vec(1,-1,1)).unit()
pao = vsr.Vec(0,0,0).null()

l3 = l2 - l1
l4 = l2.comm(l1)

scene = Scene(children=[
        frame_mesh(),
        line_mesh(l1, color='#ffff00'),
        point_mesh(pao),
        vector_mesh(pao.vec()),
        
        line_mesh(l2, color='#ffff00'),
        
        line_mesh(l3, color='powderblue'), 
        line_mesh(l4, color='deeppink'),

        vector_mesh(l_dir, position=l_loc, color=rgb2hex(255,0,123), arrow=True),
        bivector_mesh((l_loc ^ l_dir), color = 'salmon'),

        
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'gray')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

In [64]:
print(l1)
print(l2)
print(l3)
print(l4)

Dll: [ 0 0 1 0 0 0 ]
Dll: [ 0.5774 0.5774 0.5774 0 0.5774 0.5774 ]
Dll: [ 0.5774 0.5774 -0.4226 0 0.5774 0.5774 ]
Dll: [ -0.5774 0.5774 0 0 -0.5774 0.5774 ]


In [18]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
print(motor)
print(motor.trs())
print(motor.rot())

Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]
Vec: [ 1 1 1 ]
Rot: [ 0.87 0 -0.5 0 ]


In [19]:
make_text?

In [20]:
@interact(x=10,y=10,z=10)
def position_camera(x,y,z):
    camera.position = [x,y,z]

In [21]:
class Sphere(object):
    def __init__(self):
        pass
    def update(vsr_sph):
        r, p, q = vsr_sph.rpo()
        
def make_sphere():
    return Mesh(geometry = SphereGeometry(),
                material= BasicMaterial(color='red'))

def make_circle():
    return Mesh(geometry = RingGeometry(thetaSegments=64), 
              material= BasicMaterial(color='black'))

class Game(object):
    class Circle(vsr.Cir, Mesh):
        def __init__(self):
            self._color = 'black'
            super(Mesh, self).__init__(geometry = RingGeometry(thetaSegments=64), 
                          material= BasicMaterial(color=self._color))
            
        @property
        def mesh(self):
            return self._mesh
        
        def update_mesh(self):
            (r, p, q) = self.rpo()
            self.geometry.innerRadius = r
            self.geometry.outerRadius = r + 0.1
            self.position = p
            self.quaternion = q
        

## Rotation

In [22]:

n1 = vsr.Vec(0,1,0).unit()
# n2 = vsr.Vec(-1,1,0).unit()
n2 = n1.spin(vsr.Biv(-np.deg2rad(30),0,0).exp())

B = n1 ^ n2

R = n1 * n2

a = vsr.Vec(1,0,1)
a_rot = a.reflect(n2).reflect(n1)
print(a_rot)
print(a.spin(R))


scene = Scene(children=[
        frame_mesh(),
        vector_mesh(n1,color='yellow'),
        vector_mesh(n2,color='cyan'),
        vector_mesh(a, color='hotpink'),
        vector_mesh(a.reflect(n2)),
        vector_mesh(a_rot, color='black'),
#         bivector_mesh(n1.unduale(),color='yellow'),
#         bivector_mesh(n2.unduale(),color='cyan'),
        bivector_mesh(B,color='gray'),
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

AttributeError: 'Vec' object has no attribute 'reflect'

In [ ]:
vsr.Vec(1,0,0).spin(R)

In [ ]:
2 * np.rad2deg(np.arccos(R[0]))

In [ ]:
sph1 = vsr.Sph(vsr.Vec(0.7,0,0).null(), 1.5)
sph2 = vsr.Sph(vsr.Vec(-0.5,0,0).null(), 1.)
cir = sph1.meet(sph2)
r = cir.rot()

sphere1 = Mesh(geometry = SphereGeometry(radius=sph1.radius(),
                                        widthSegments=64,
                                        heightSegments=64), 
              material= BasicMaterial(color='red'),
              position=np.array(sph1.pnt())[:3].tolist())

sphere2 = Mesh(geometry = SphereGeometry(radius=sph2.radius(),
                                        widthSegments=64,
                                        heightSegments=64), 
              material= BasicMaterial(color='blue', wireframe=False),
              position=np.array(sph2.pnt())[:3].tolist())

circle = Mesh(geometry = RingGeometry(innerRadius=cir.radius(),
                                                outerRadius=cir.radius()+0.05,
                                                thetaSegments=64), 
              material= BasicMaterial(color='black'),
              position=np.array(cir.pnt())[:3].tolist(),
              quaternion = cir.rot().quat().tolist())



scene = Scene(children=[sphere1, sphere2, circle])
c = PerspectiveCamera(position=[10, 5, 10])
renderer = Renderer(camera=c, scene = scene, controls=[TrackballControls(controlling=c)],
                    background = 'white')
display(renderer)
@interact(circlecolor=('hotpink', 'gray', 'black'),
         sphere1color=('hotpink', 'gray', 'black'),
         sphere2color=('hotpink', 'gray', 'black'))
def set_radius(circlecolor, sphere1color, sphere2color):
    sphere1.material.color = sphere1color
    sphere2.material.color = sphere2color
    circle.material.color = circlecolor

In [ ]:
%%javascript
console.log('hallo')
Jupyter.notebook.get_selected_cell()

In [ ]:
cir = vsr.Sph(vsr.Vec(0,0,0).null(), 3).meet(vsr.Sph(vsr.Vec(1,0,0).null(), 3))
print(cir.rpo())
radius, position, quat = cir.rpo()